In [5]:
#pip install --user scikit-image
#pip install --user keras
#pip install --user opencv-python-headless

import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches


# Root directory of the Mask_RCNN project
ROOT_DIR = os.path.abspath("/home/idies/workspace/Storage/ncarey/persistent/PARADIM/furnace_ml/Mask_RCNN")
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log
import keras.backend
import paradim

#%matplotlib inline 


# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
PARADIM_WEIGHTS_PATH = "/home/idies/workspace/paradim_data/models/summer_school_model.h5"  # TODO: update this path
#PARADIM_WEIGHTS_PATH = "/home/idies/workspace/Storage/ncarey/persistent/PARADIM/furnace_ml/trained_models/mask_rcnn_paradim_0020_bigtrain.h5" 
#PARADIM_WEIGHTS_PATH = "/home/idies/workspace/Storage/ncarey/persistent/PARADIM/furnace_ml/Mask_RCNN/logs/paradim20190609T0213/mask_rcnn_paradim_0020.h5"  # TODO: update this path

PARADIM_DIR = "/home/idies/workspace/Storage/ncarey/persistent/PARADIM/furnace_ml/datasets/paradim"


#main
import time
import os
import cv2 #pip install --user opencv-python-headless  
#it is important to install the headless version as there is a missing shared lib for the regular opencv-python
from SciServer import CasJobs


my_context = "MyDB"
path_prefix = '/home/idies/workspace'
to_process_path = '/home/idies/workspace/Storage/ncarey/persistent/PARADIM/furnace_ml/datasets/paradim/current/goodMelt/frame.jpg'
CROP_DIR = '/home/idies/workspace/paradim_data/frameClass/croppedFrames'
CLASSIFIED_DIR = '/home/idies/workspace/paradim_data/frameClass/classifiedFrames'

In [6]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax


class ParadimClassifier:
    
    def classifyCurrent(self, i, classified_frame_path):

        image_id = random.choice(self.dataset.image_ids)
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(self.dataset, self.config, image_id, use_mini_mask=False)
        info = self.dataset.image_info[image_id]
        #print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
        #                               self.dataset.image_reference(image_id)))

        # Run object detection
        results = self.model.detect([image], verbose=1)

        # Display results
        
        #close prvious results?
        plt.close()
        
        r = results[0]
        visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            self.dataset.class_names, r['scores'], #ax=self.ax,
                            title="Predictions", path_to_save = classified_frame_path)
        

        #self.fig.canvas.draw()
        #plt.show()
        #log("gt_class_id", gt_class_id)
        #log("gt_bbox", gt_bbox)
        #log("gt_mask", gt_mask)
        return r
    
    def __init__(self, weights_path, dataset_dir, model_dir):
        self.config = paradim.ParadimConfig()
        # Override the training configurations with a few
        # changes for inferencing.
        class InferenceConfig(self.config.__class__):
            # Run detection on one image at a time
            GPU_COUNT = 1
            IMAGES_PER_GPU = 1
            DETECTION_MIN_CONFIDENCE = .4
            IMAGE_RESIZE_MODE="square"

        self.config = InferenceConfig()
        self.config.display()
        # Device to load the neural network on.
        DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0
        TEST_MODE = "inference"

        # Load validation dataset
        self.dataset = paradim.ParadimDataset()
        self.dataset_dir = dataset_dir
        self.dataset.load_paradim(self.dataset_dir, "current")

        # Must call before using the dataset
        self.dataset.prepare()

        #print("Images: {}\nClasses: {}".format(len(self.dataset.image_ids), self.dataset.class_names))
        
        #lil hack to make it work as specified in 
        #https://github.com/matterport/Mask_RCNN/issues/30
        K = keras.backend.backend()
        if K=='tensorflow':
            keras.backend.set_image_dim_ordering('tf')

        self.model_dir = model_dir
        
        with tf.device(DEVICE):
            self.model = modellib.MaskRCNN(mode="inference", model_dir=self.model_dir, config=self.config)
            
        self.weights_path = weights_path

        # Or, load the last model you trained
        #weights_path = model.find_last()

        # Load weights
        print("Loading weights ", self.weights_path)
        self.model.load_weights(self.weights_path, by_name=True)
        
        #self.fig, self.ax = plt.subplots(1,1)


In [9]:
def frameCrop(path, hotseat_path, crop_path):
    xStartPct = .65 
    xStopPct = 0.95
    yStartPct = .1
    yStopPct = .5
    
    img = cv2.imread(path)
    width, height, pixels = img.shape
    crop_img = img[(int)(width*yStartPct):(int)(width*yStopPct),(int)(height*xStartPct):(int)(height*xStopPct)].copy()
    cv2.imwrite(crop_path, crop_img)
    cv2.imwrite(hotseat_path, crop_img)


classifier = ParadimClassifier(weights_path = PARADIM_WEIGHTS_PATH, dataset_dir = PARADIM_DIR, model_dir = MODEL_DIR)

        
        
    



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.4
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  576
IMAGE_META_SIZE                16
IMAGE_MIN_DIM                  432
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [576 576   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_bbox_loss': 1.0, 'rpn_class_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_mask_loss': 1.0, 'mrcnn_bbox_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE             

ValueError: Layer #391 (named "mrcnn_bbox_fc"), weight <tf.Variable 'mrcnn_bbox_fc_2/kernel:0' shape=(1024, 16) dtype=float32_ref> has shape (1024, 16), but the saved weight has shape (1024, 12).

In [8]:
count = 0
while True: #really should be while true. Will change in future, I just didnt want it running indefinitely waiting for a CTRL+C
    count = count + 1
    #time.sleep(.1)
    
    checkup_query = "select * from FramePathProcessing where processed = 0 ORDER BY Id"
    print("searching for unprocessed...")
    unprocessed_df = CasJobs.executeQuery(sql=checkup_query, context=my_context)
    if len(unprocessed_df['Path']) == 0:
        time.sleep(1)
    
    for unprocessed_path in unprocessed_df['Path']: 
        full_path = os.path.join(path_prefix, unprocessed_path)
        base_name = os.path.basename(full_path)
        crop_path = os.path.join(CROP_DIR, base_name)
        frameCrop(full_path, to_process_path, crop_path)
        classified_frame_path = os.path.join(CLASSIFIED_DIR, base_name)
        
        
        #start classification
        r = classifier.classifyCurrent(count, classified_frame_path)
        #update DB stating that the image has been processed (change processed from 0 to 1 in relevant row)
        #print(r)
        print(r['scores'])
        goodMeltScore = 0
        fastBottomScore = 0
        fastTopScore = 0
        for index in range(len(r['class_ids'])):
            cur_class_id = r['class_ids'][index]
            cur_class_score = r['scores'][index]
            #need to fix multiple scores of same class...
            if cur_class_id == 1:
                goodMeltScore = cur_class_score
            elif cur_class_id == 2:
                fastBottomScore = cur_class_score
            elif cur_class_id == 3:
                fastTopScore = cur_class_score
            else:
                print("ERROR, UNRECOGNIZED CLASS ID FROM MR_CNN CLASSIFIER")
            
        # TODO this would be faster if i insert via ID instead of Path
        update_sql = '''UPDATE FramePathProcessing
                        SET processed = 1, goodMeltScore = {0}, fastBottomScore = {1}, fastTopScore = {2}, processedPath = '{3}', cropPath = '{4}'
                        WHERE Path = '{5}' '''.format(goodMeltScore, fastBottomScore, fastTopScore, classified_frame_path, crop_path, unprocessed_path)
        print(update_sql)
        CasJobs.executeQuery(sql=update_sql, context=my_context)


searching for unprocessed...


NameError: name 'classifier' is not defined

In [ ]:
#DEBUGGING, DONT RUN THIS

checkup_query = "select * from FramePathProcessingFirstRun where processed = 0 ORDER BY Id"
print("searching for unprocessed...")
unprocessed_df = CasJobs.executeQuery(sql=checkup_query, context=my_context)
count = 1
for unprocessed_path in unprocessed_df['Path'][6:]: 
    full_path = os.path.join(path_prefix, unprocessed_path)
    base_name = os.path.basename(full_path)
    crop_path = os.path.join(CROP_DIR, base_name)
    frameCrop(full_path, to_process_path, crop_path)
    classified_frame_path = os.path.join(CLASSIFIED_DIR, base_name)
        
        
    #start classification
    r = classifier.classifyCurrent(count, classified_frame_path)
    #update DB stating that the image has been processed (change processed from 0 to 1 in relevant row)
    #print(r)
    print(classified_frame_path)
    print(r['scores'])
    break